# Convert between Lat/Long/Height and ECEF XYZ



In [ ]:
include("../modules/geometry.jl")

#example Lat/Long/Height
geo=[35.38987,-111.8116,9748.89523]
println("GEO: ", geo)

#compute ECEF XYZ based LLH
xyz = Geometry.geo_to_xyz(geo)
println("GEO -> XYZ: ", xyz)

#convert XYZ back to LLH
geo = Geometry.xyz_to_geo(xyz)
println("GEO -> XYZ -> GEO", geo)

#convert lots of LLH points to XYZ
@time xyz = Geometry.geo_to_xyz(repeat(geo, 1, 1000))

#convert lots of XYZ points to LLH
@time geo = Geometry.xyz_to_geo(xyz)


# Convert SCH coordinates to Approximate ECEF XYZ 

In [ ]:
include("../modules/geometry.jl")

#create PEG point based on (lat,long,heading) (peg point is origin of SCH CS)
peg = Geometry.PegPoint(35.2117072245, -111.8112805579, 179.8535529463)

#example SCH coordinate
sch=[-19766.4,23.145535442,9748.895229822]

#convert SCH coordinate to XYZ
xyz = Geometry.sch_to_xyz(sch, peg)
println("SCH -> XYZ", xyz)

#convert lots of SCH to XYZ. This would be how one would convert an sch grid to an XYZ grid
@time xyz = Geometry.sch_to_xyz(repeat(sch,1, 1000), peg)

# Working with Quaternions for vector and frame rotations

In [ ]:
include("../modules/geometry.jl")

#create quaternion q(θ, v) to describe rotation of θ (degrees), about vector v (unit)
q = Geometry.quat(45, [0,1,0]) #create a quaternion to rotate a vector by 45 degrees about yaxis [0,1,0]

#rotate a vector using the quaternion
vec = [1,0,0];
#rotate the vector
rotated_vec = Geometry.rotate_vec([1,0,0], q) #rotate a vector aligned with the x-axis, by q
println("Original vector: ", vec)
println("Rotated Vector: ", rotated_vec)


## Combined Vector Rotations
- Vector rotations can be combined by simply multiplying quaternions. 
- The convention chosen here enforces left multiplication order
    - ie. qr = q1 * q2 means rotate about q2 first then q1

In [ ]:
#create three rotation quaternions
qx = Geometry.quat(90, [1,0,0]) #create a quaternion to rotate a vector by 90 degrees about xaxis
qy = Geometry.quat(90, [0,1,0]) #create a quaternion to rotate a vector by 90 degrees about yaxis
qz = Geometry.quat(90, [0,0,1]) #create a quaternion to rotate a vector by 90 degrees about zaxis

#vector to experiment with
vec = [0.,1.,0.]
println("Original Vector (no rotation): ", vec)


#combine rotations using qy then qx (order is outside in)
rotated_vec = Geometry.rotate_vec(vec, qy) #rotate a vector aligned with the x-axis, by qy*qx
println("\nRotated Vector (qy): ", rotated_vec)
rotated_vec = Geometry.rotate_vec(vec, qx*qy) #rotate a vector aligned with the x-axis, by qy*qx
println("Rotated Vector (qx*qy): ", rotated_vec)


#combine rotations using qx then qy
rotated_vec = Geometry.rotate_vec(vec, qx) #rotate a vector aligned with the x-axis, by qy*qx
println("\nRotated Vector, vr = v (*) qx: ", rotated_vec)
rotated_vec = Geometry.rotate_vec(rotated_vec, qy) #rotate a vector aligned with the x-axis, by qy*qx
println("Rotated Vector, vrr = vr (*) qy: ", rotated_vec)
rotated_vec = Geometry.rotate_vec(vec, qy*qx) #rotate a vector aligned with the x-axis, by qy*qx
println("Rotated Vector vr = v (*) qy*qx: ", rotated_vec)


#all combinations of the three rotations
rotated_vec = Geometry.rotate_vec(vec, qx*qy*qz) #rotate a vector aligned with the x-axis, by qx*qy*qz
println("\nCombined Rotated Vector, qx*qy*qz: ", rotated_vec)
rotated_vec = Geometry.rotate_vec(vec, qx*qz*qy) #rotate a vector aligned with the x-axis, by qx*qz*qy
println("Combined Rotated Vector, qx*qz*qy: ", rotated_vec)
rotated_vec = Geometry.rotate_vec(vec, qy*qx*qz) #rotate a vector aligned with the x-axis, by qy*qx*qz
println("Combined Rotated Vector, qy*qx*qz: ", rotated_vec)
rotated_vec = Geometry.rotate_vec(vec, qy*qz*qx) #rotate a vector aligned with the x-axis, by qy*qz*qx
println("Combined Rotated Vector, qy*qz*qx: ", rotated_vec)
rotated_vec = Geometry.rotate_vec(vec, qz*qy*qx) #rotate a vector aligned with the x-axis, by qz*qy*qx
println("Combined Rotated Vector, qz*qy*qx: ", rotated_vec)
rotated_vec = Geometry.rotate_vec(vec, qz*qx*qy) #rotate a vector aligned with the x-axis, by qz*qx*qy
println("Combined Rotated Vector, qz*qx*qy: ", rotated_vec)


# Frame Rotations (or vector projection)
The frame a vector is described in is rotated, resulting in a modified description of the original vector (in other words, projection). 
This ends up being the opposite operation of a vector rotation 

In [ ]:
# rotate a frame, then describe vector in the rotated frame

#define vector to project 
vec = [1.0, 0.0, 0.0]
println("\nOrignial Vector: ", vec);

#create frame rotation quaternions 
qy = Geometry.quat(90, [0,1,0]) #create a quaternion to rotate a frame by 90 degrees about yaxis [0,1,0]
qz= Geometry.quat(90, [0,0,1]) #create a quaternion to rotate a frame by 90deg about z-axis

#project and print vectors
projected_vec = Geometry.rotate_frame(vec, qy) #project a vector aligned with the x-axis, to a rotated frame described by q
println("Projected Vector: ", projected_vec)

projected_vec = Geometry.rotate_frame(vec, qz) #project a vector aligned with the x-axis, to a rotated frame described by q
println("Projected Vector: ", projected_vec)


## Combining rotations for Frame Rotations
Quaternion multiplication can be thought of in two ways, extrinsic vs. intrinsic rotations:
 - Intrinsic rotation: second rotation is about rotated frame
    + q = q1*q2 = rotate q1 and then rotate q2 about rotated frame (intrinsic)
 - extrinsic rotation: second rotation is about the original frmae
    + q = q2*q1 = rotate q1 and then rotate q2 about original frame (extrinsic)

In [44]:
include("../modules/geometry.jl")

qy = Geometry.quat(90, [0,1,0]) #create a quaternion to rotate a frame by 90 degrees about yaxis [0,1,0]
qz= Geometry.quat(90, [0,0,1]) #create a quaternion to rotate a frame by 90deg about z-axis

#original vector
vec = [0.0,0.0,1.0];

#Intrinsic Rotation Example
#Rotate about z-axis, to get x'y'z', then rotate about y' axis
println("Intrinsic projected vector [zero rotations]: ", vec)
projected_vec = Geometry.rotate_frame(vec, qz) 
println("Intrinsic projected vector [first rotation]: ", projected_vec)
projected_vec = Geometry.rotate_frame(vec, qz*qy) 
println("Intrinsic projected vector [second rotation]: ", projected_vec)

#Extrinsic Rotation Example
println("\nExtrinsic projected vector [zero rotations]: ", vec)
projected_vec = Geometry.rotate_frame(vec, qz) 
println("Extrinsic projected vector [first rotation]: ", projected_vec)
projected_vec = Geometry.rotate_frame(vec, qy*qz) 
println("Extrinsic projected vector [second rotation]: ", projected_vec)


Intrinsic projected vector [zero rotations]: [0.0, 0.0, 1.0]
Intrinsic projected vector [first rotation]: [0.0, 0.0, 1.0]
Intrinsic projected vector [second rotation]: [-1.0, 0.0, 0.0]

Extrinsic projected vector [zero rotations]: [0.0, 0.0, 1.0]
Extrinsic projected vector [first rotation]: [0.0, 0.0, 1.0]
Extrinsic projected vector [second rotation]: [0.0, 1.0, 0.0]
